# DB2-Salesforce connector: Citation information updates

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Record_ID__c'
object_id = 'nanoHUB_citations__c'

# login parameters to be handled by Papermill
# login credentials
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"purdue99272844vp8Z6qq0teuPQmirGaaPAt4",    
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [3]:
import sys
import os
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import datetime

## Obtain tool information from DB2 

In [4]:
# Basic tool information
sql_query = "select *, c.id as c_id from jos_citations as c \
            left join jos_citations_types as t \
            on c.type = t.id \
            where uid=2143"

# display
print(sql_query)

select *, c.id as c_id from jos_citations as c             left join jos_citations_types as t             on c.type = t.id             where uid=2143


In [5]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))
df = pd.read_sql_query(sql_query, engine)

In [6]:
# convert and clean some columns, so we can, for example, sort values
df['date_publish'] = pd.to_datetime(df['date_publish'], errors='coerce') 
df['created'] = pd.to_datetime(df['created'], errors='coerce') 
df['date_submit'] = pd.to_datetime(df['date_submit'], errors='coerce') 
df['date_accept'] = pd.to_datetime(df['date_accept'], errors='coerce') 

In [7]:
# display
display(df.head(1))
display('size='+str(df.shape))

,id,uid,affiliated,fundedby,created,address,author,booktitle,chapter,cite,...,custom2,custom3,custom4,pdf_url,id,type,type_title,type_desc,type_export,c_id
0,2036,2143,1,-1,NaT,,S. Datta{{8629}},,,,...,None,None,None,https://nanohub.org/nanoHUBCitations/citmanage...,3,proceedings,Proceedings,None,None,2036


'size=(2467, 68)'

## Match data with Salesforce format

In [8]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here

df_sf['Record_ID__c']      = df['c_id']
df_sf['URL__c']            = df['url']
df_sf['NCN_Affiliated__c'] = df['affiliated'].apply(lambda x: 1 if x==1 else 0)
df_sf['Author_List__c']    = df['author']
df_sf['Journal__c']        = df['journal']
df_sf['Year__c']           = df['year']
df_sf['Type__c']           = df['ref_type']
df_sf['Notes__c']          = df['notes']
df_sf['Name']              = df['title'].apply(lambda x: x[:70])
df_sf['Full_title__c']     = df['title']
df_sf['Format__c']         = df['type_title']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Record_ID__c,2036,1483
URL__c,http://scholar.google.com/scholar?hl=en&q=Non-...,http://scholar.google.com/scholar?hl=en&q=http...
NCN_Affiliated__c,1,1
Author_List__c,S. Datta{{8629}},Kelsey Rodgers{{68942}};Heidi Diiefes-Dux{{641...
Journal__c,IWCE 2015 Proceedings,
Year__c,2015,2013
Type__c,R,"R,E"
Notes__c,{{IEEE}},This is a draft paper that has been accepted b...
Name,Non-equilibrium Green's Function \NEGF) Method...,Case Studies: First-Year Engineering Nanotechn...
Full_title__c,Non-equilibrium Green's Function \NEGF) Method...,Case Studies: First-Year Engineering Nanotechn...


## To Salesforce Sales Cloud CRM

In [10]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [11]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000MuDf3AAF
hello
[Success] CSV upload successful. Job ID = 7505w00000MuDf3AAF
[Success] Closing job successful. Job ID = 7505w00000MuDf3AAF


In [12]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000MuDf3AAF',
 'operation': 'upsert',
 'object': 'nanoHUB_citations__c',
 'createdById': '0055w00000ArpYvAAJ',
 'createdDate': '2020-06-13T02:34:15.000+0000',
 'systemModstamp': '2020-06-13T02:34:18.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Record_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [13]:
from pprint import pprint
pprint(db_s.check_bulk_failed_results())

''
